In [1]:
# load 1 daatset first
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# https://www.kaggle.com/code/ysthehurricane/step-by-step-huggingface-model-fine-tune-guide

#Llama2 is too big :( and trying to use it made me run out of memory very fast so we are using a kaggle guide to finetune a model to the data

In [3]:


import keras
import sklearn
import pandas as pd
import numpy as np
from keras import preprocessing
from keras import models
from keras import layers
!pip install pytorch_lightning
!ls drive/'My Drive'/'Colab Notebooks'/'DATA'/'exoplanets.csv'

df = pd.read_csv("drive/My Drive/Colab Notebooks/DATA/exoplanets.csv", header=[0])

dfvals = df.values

X = dfvals[:, :-1]
y = dfvals[:, -1]
df.head()

'drive/My Drive/Colab Notebooks/DATA/exoplanets.csv'


,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,10811496,K00753.01,NaN,CANDIDATE,CANDIDATE,0.000,0,0,0,0,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,10848459,K00754.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,10854555,K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [4]:
# prompt: git clone https://huggingface.co/meta-llama/Llama-2-7b

from google.colab import drive
import keras
import sklearn
import pandas as pd
import numpy as np
from keras import preprocessing
from keras import models
from keras import layers

# load 1 dataset first

drive.mount('/content/drive')

!ls drive/'My Drive'/'Colab Notebooks'/'DATA'/'exoplanets.csv'

# read first dataset
df = pd.read_csv("drive/My Drive/Colab Notebooks/DATA/exoplanets.csv", header=[0])

dfvals = df.values

X = dfvals[:, :-1]
y = dfvals[:, -1]

df.head()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'drive/My Drive/Colab Notebooks/DATA/exoplanets.csv'


,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,10811496,K00753.01,NaN,CANDIDATE,CANDIDATE,0.000,0,0,0,0,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,10848459,K00754.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,10854555,K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [5]:

from huggingface_hub import notebook_login

#notebook_login()

In [6]:

from sklearn.model_selection import train_test_split


import torch

from transformers import T5Tokenizer
from transformers import T5ForConditionalGeneration, AdamW

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

pl.seed_everything(100)

import warnings
warnings.filterwarnings("ignore")
print("Number of records: ", df.shape[0])
# Apply .str.lower() to all string columns in the DataFrame
for column in df.select_dtypes(include="object").columns:
    df[column] = df[column].str.lower()


INFO:lightning_fabric.utilities.seed:Seed set to 100


Number of records:  9564


In [7]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
INPUT_MAX_LEN = 512 # Input length
OUT_MAX_LEN = 128 # Output Length
TRAIN_BATCH_SIZE = 8 # Training Batch Size
VALID_BATCH_SIZE = 2 # Validation Batch Size
EPOCHS = 5 # Number of Iteration

MODEL_NAME = "t5-base"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME, model_max_length= INPUT_MAX_LEN)
print("eos_token: {} and id: {}".format(tokenizer.eos_token, tokenizer.eos_token_id)) # End of token (eos_token)
print("unk_token: {} and id: {}".format(tokenizer.unk_token, tokenizer.eos_token_id)) # Unknown token (unk_token)
print("pad_token: {} and id: {}".format(tokenizer.pad_token, tokenizer.eos_token_id)) # Pad token (pad_token)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


eos_token: </s> and id: 1
unk_token: <unk> and id: 1
pad_token: <pad> and id: 1


In [ ]:
class T5Dataset:
    def __init__(self, input_data, target_data):
        self.input_data = input_data
        self.target_data = target_data
        self.tokenizer = tokenizer
        self.input_max_len = INPUT_MAX_LEN
        self.out_max_len = OUT_MAX_LEN

    def __len__(self):
        return len(self.input_data)

    def __getitem__(self, item):
        input_text = str(self.input_data[item])
        target_text = str(self.target_data[item])

        inputs_encoding = self.tokenizer(
            input_text,
            add_special_tokens=True,
            max_length=self.input_max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt"
        )

        output_encoding = self.tokenizer(
            target_text,
            add_special_tokens=True,
            max_length=self.out_max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt"
        )

        inputs_ids = inputs_encoding["input_ids"].flatten()
        attention_mask = inputs_encoding["attention_mask"].flatten()
        labels = output_encoding["input_ids"]

        labels[labels == 0] = -100  # Mask padding tokens as per T5 documentation

        labels = labels.flatten()

        return {
            "input_ids": inputs_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }


In [ ]:
class T5DatasetModule(pl.LightningDataModule):
    def __init__(self, train_input, train_target, val_input, val_target):
        super().__init__()
        self.train_input = train_input
        self.train_target = train_target
        self.val_input = val_input
        self.val_target = val_target
        TRAIN_BATCH_SIZE = 4  #smaller
        VALID_BATCH_SIZE = 4 


    def setup(self, stage=None):
        self.train_dataset = T5Dataset(self.train_input, self.train_target)
        self.val_dataset = T5Dataset(self.val_input, self.val_target)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=16, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=16)


In [ ]:
class T5Model(pl.LightningModule):

    def __init__(self):
        super().__init__()
        self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        return output.loss, output.logits

    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]  

        loss, outputs = self(input_ids, attention_mask, labels)

        self.log("train_loss", loss, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"] 

        loss, outputs = self(input_ids, attention_mask, labels)

        self.log("val_loss", loss, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=0.0001)


In [ ]:
from torch.utils.data import DataLoader

def preprocess_dataframe(df):
    """
    Preprocess the DataFrame by combining all columns into a single 'input' column,
    and optionally generating a 'target' column.
    """
    df = df.fillna("none")

    for column in df.select_dtypes(include="object").columns:
        df[column] = df[column].apply(lambda x: " ".join(str(x).split()) if isinstance(x, str) else str(x))

    # combine columns into a single text field
    df["input"] = df.apply(lambda row: " ".join(row.astype(str)), axis=1)

    
    df["target"] = df["input"]  

    return df

def run():
    preprocessed_df = preprocess_dataframe(df)


    df_train, df_valid = train_test_split(preprocessed_df[0:10000], test_size=0.2, random_state=101)

    df_train = df_train.reset_index(drop=True)
    df_valid = df_valid.reset_index(drop=True)


    dataModule = T5DatasetModule(df_train["input"], df_train["target"], df_valid["input"], df_valid["target"])
    dataModule.setup()

    # device and model
    device = DEVICE
    models = T5Model()
    models.to(device)

    # checkpointing
    checkpoint_callback = ModelCheckpoint(
        dirpath="/kaggle/working",
        filename="best_checkpoint",
        save_top_k=2,
        verbose=True,
        monitor="val_loss",
        mode="min"
    )

    devices = [0] if torch.cuda.is_available() else ["cpu"]

    trainer = pl.Trainer(
    callbacks=[checkpoint_callback],
    max_epochs=2,
    devices=1,
    accelerator="gpu",
    precision = 16,
    accumulate_grad_batches=4,
)




    # Train 
    trainer.fit(models, dataModule)

run()


INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params | Mode
------------------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M  | eval
------------------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)
0         Modules in train mode
541       Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 96.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 29.06 MiB is free. Process 83238 has 14.72 GiB memory in use. Of the allocated memory 14.35 GiB is allocated by PyTorch, and 239.74 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
'''
#!git clone https:'''//huggingface.co/meta-llama/Llama-2-7b
# Model from Hugging Face hub
base_model = "meta-llama/Llama-2-7b-chat-hf"


# Fine-tuned model
new_model = "llama-2-7b-chat-space"

'''

In [15]:
#load model
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
'''
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
# Add a padding token to the tokenizer
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Load model with torch_dtype specified
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", torch_dtype=torch.float16)

# Assume 'kepoi_name' column contains the text you want to tokenize
def tokenize_function(example):
    # Access the 'kepoi_name' column for tokenization
    # example is now a string from the "kepoi_name" column
    # Ensure all token IDs are within the valid vocabulary range
    max_length = 128  # You can adjust this value based on your data and model

    tokens = tokenizer(
        example,
        padding="max_length",
        truncation=True,
        max_length=max_length  # Add max_length here
    )
    tokens['input_ids'] = [id % tokenizer.vocab_size for id in tokens['input_ids']]  # Wrap out-of-range IDs
    return tokens

# Apply to the 'kepoi_name' column of the DataFrame
tokenized_data = df['kepoi_name'].map(tokenize_function)

# Convert the tokenized data to a DataFrame if needed
tokenized_datasets = pd.DataFrame(list(tokenized_data))
'''

'\ntokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")\n# Add a padding token to the tokenizer\ntokenizer.add_special_tokens({\'pad_token\': \'[PAD]\'})\n\n# Load model with torch_dtype specified\nmodel = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", torch_dtype=torch.float16) \n\n# Assume \'kepoi_name\' column contains the text you want to tokenize\ndef tokenize_function(example):\n    # Access the \'kepoi_name\' column for tokenization\n    # example is now a string from the "kepoi_name" column\n    # Ensure all token IDs are within the valid vocabulary range\n    max_length = 128  # You can adjust this value based on your data and model\n\n    tokens = tokenizer(\n        example, \n        padding="max_length", \n        truncation=True, \n        max_length=max_length  # Add max_length here\n    )\n    tokens[\'input_ids\'] = [id % tokenizer.vocab_size for id in tokens[\'input_ids\']]  # Wrap out-of-range IDs\n    return tokens\n\n# A